# 利用 trtexec 导出 engine 并进行推理 输入图片尺寸 320x320

In [1]:
!/usr/src/tensorrt/bin/trtexec --onnx=yolov5n.trt.onnx --saveEngine=yolov5n_320_trtexec.engine --workspace=8192

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=yolov5n.trt.onnx --saveEngine=yolov5n_320_trtexec.engine --workspace=8192
[03/11/2022-21:31:30] [I] === Model Options ===
[03/11/2022-21:31:30] [I] Format: ONNX
[03/11/2022-21:31:30] [I] Model: yolov5n.trt.onnx
[03/11/2022-21:31:30] [I] Output:
[03/11/2022-21:31:30] [I] === Build Options ===
[03/11/2022-21:31:30] [I] Max batch: explicit batch
[03/11/2022-21:31:30] [I] Workspace: 8192 MiB
[03/11/2022-21:31:30] [I] minTiming: 1
[03/11/2022-21:31:30] [I] avgTiming: 8
[03/11/2022-21:31:30] [I] Precision: FP32
[03/11/2022-21:31:30] [I] Calibration: 
[03/11/2022-21:31:30] [I] Refit: Disabled
[03/11/2022-21:31:30] [I] Sparsity: Disabled
[03/11/2022-21:31:30] [I] Safe mode: Disabled
[03/11/2022-21:31:30] [I] DirectIO mode: Disabled
[03/11/2022-21:31:30] [I] Restricted mode: Disabled
[03/11/2022-21:31:30] [I] Save engine: yolov5n_320_trtexec.engine
[03/11/2022-21:31:30] [I] Load engine: 
[03/11/2022-21:31:30] 

[03/11/2022-21:31:33] [I] [TRT] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +146, now: CPU 446, GPU 3008 (MiB)
[03/11/2022-21:31:35] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +240, GPU +246, now: CPU 686, GPU 3254 (MiB)
[03/11/2022-21:31:35] [I] [TRT] Local timing cache in use. Profiling results in this builder pass will not be stored.
[03/11/2022-21:33:57] [I] [TRT] Detected 1 inputs and 4 output network tensors.
[03/11/2022-21:33:57] [I] [TRT] Total Host Persistent Memory: 154592
[03/11/2022-21:33:57] [I] [TRT] Total Device Persistent Memory: 6233088
[03/11/2022-21:33:57] [I] [TRT] Total Scratch Memory: 12096768
[03/11/2022-21:33:57] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 2 MiB, GPU 535 MiB
[03/11/2022-21:33:57] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took 49.5465ms to assign 7 blocks to 112 nodes requiring 16513536 bytes.
[03/11/2022-21:33:57] [I] [TRT] Total Activation Memory: 16513536
[03/11/2022-21:33:57] [I] [TRT] 

In [2]:
import os
import torch
import cv2
import numpy as np

cuda_visible = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_visible

assert torch.cuda.is_available()
device = torch.device('cuda')

import tensorrt as trt
print(f"We're using TensorRT: {trt.__version__} on {device} device: {cuda_visible}.")

We're using TensorRT: 8.2.1.8 on cuda device: 0.


In [3]:
# engine_path = "yolov5n.engine"
engine_path = "yolov5n_320_trtexec.engine"
img_size = 320
img_path = "../test/assets/bus.jpg"
img = cv2.imread(img_path)
img = cv2.resize(img,(img_size,img_size))
img = img.astype(np.float32)
img /= 255
img = np.transpose(img,(2,0,1))
# img = np.expand_dims(img, axis=0)
img = np.ascontiguousarray(img)

In [4]:
img_tensor = torch.from_numpy(img).to(device)
img_tensor.shape

torch.Size([3, 320, 320])

In [5]:
from yolort.runtime import PredictorTRT

# 测试 trtexec 导出的 engine 推理速度

In [6]:
y_runtime = PredictorTRT(engine_path, device=device)
y_runtime.warmup()

Loading yolov5n_320_trtexec.engine for TensorRT inference...


In [7]:
%%timeit
predictions_trt = y_runtime.predict(img_tensor)

19.5 ms ± 364 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 测试 tensorrt 导出的 engine 推理速度(未完成)

In [ ]:
y_runtime = PredictorTRT(engine_path, device=device)
y_runtime.warmup()

In [ ]:
predictions_trt = y_runtime.predict(img_tensor)